In [1]:
dbutils.widgets.text("tenant", "", "Tenant")
tenant = dbutils.widgets.get("tenant")
if tenant == "":
  raise Exception

dbutils.widgets.text("env", "", "Environment")
env = dbutils.widgets.get("env")
if env == "":
  raise Exception

In [2]:
from azure.storage.filedatalake import DataLakeServiceClient
service_client = DataLakeServiceClient(account_url="{}://{}.dfs.core.windows.net".format(
        "https", dbutils.secrets.get(scope="dgsecretscope", key="storageadlsgen2name")),
        credential=dbutils.secrets.get(scope="dgsecretscope", key="storageaccesskey"))

In [3]:
fs_client = service_client.create_file_system(tenant)
fs_client.create_directory("code")
fs_client.create_directory("data")
fs_client.create_directory("logs")

fs_client.create_directory("data/databases")
fs_client.create_directory("data/databases/dg_{}".format(tenant))
fs_client.create_directory("data/raw")
fs_client.create_directory("data/pb_datasets")
fs_client.create_directory("data/adhoc")

--------------------------------------------------------------------------- 
 StorageErrorException Traceback (most recent call last)
 /databricks/python/lib/python3.7/site-packages/azure/storage/blob/_container_client.py in create_container (self, metadata, public_access, **kwargs) 
 286 headers = headers , 
 --> 287 **kwargs)
 288 except StorageErrorException as error : 

 /databricks/python/lib/python3.7/site-packages/azure/storage/blob/_generated/operations/_container_operations.py in create (self, timeout, metadata, access, request_id, container_cpk_scope_info, cls, **kwargs) 
 119 map_error ( status_code = response . status_code , response = response , error_map = error_map ) 
 --> 120 raise models . StorageErrorException ( response , self . _deserialize ) 
 121 

 StorageErrorException : Operation returned an invalid status 'The specified container already exists.'

During handling of the above exception, another exception occurred:

 ResourceExistsError Traceback (most recent call last)
 <command-3465384727029774> in <module> 
 ----> 1 fs_client = service_client . create_file_system ( tenant ) 
 2 fs_client . create_directory ( "code" ) 
 3 fs_client . create_directory ( "data" ) 
 4 fs_client . create_directory ( "logs" ) 
 5 

 /databricks/python/lib/python3.7/site-packages/azure/storage/filedatalake/_data_lake_service_client.py in create_file_system (self, file_system, metadata, public_access, **kwargs) 
 244 """
 245 file_system_client = self . get_file_system_client ( file_system ) 
 --> 246 file_system_client . create_file_system ( metadata = metadata , public_access = public_access , ** kwargs ) 
 247 return file_system_client
 248 

 /databricks/python/lib/python3.7/site-packages/azure/storage/filedatalake/_file_system_client.py in create_file_system (self, metadata, public_access, **kwargs) 
 241 return self._container_client.create_container(metadata=metadata,
 242 public_access = public_access , 
 --> 243 **kwargs)
 244 
 245 def delete_file_system ( self , ** kwargs ) : 

 /databricks/python/lib/python3.7/site-packages/azure/core/tracing/decorator.py in wrapper_use_tracer (*args, **kwargs) 
 81 span_impl_type = settings . tracing_implementation ( ) 
 82 if span_impl_type is None : 
 ---> 83 return func ( * args , ** kwargs ) 
 84 
 85 # Merge span is parameter is set, but only if no explicit parent are passed 

 /databricks/python/lib/python3.7/site-packages/azure/storage/blob/_container_client.py in create_container (self, metadata, public_access, **kwargs) 
 287 **kwargs)
 288 except StorageErrorException as error : 
 --> 289 process_storage_error ( error ) 
 290 
 291 @ distributed_trace

 /databricks/python/lib/python3.7/site-packages/azure/storage/blob/_shared/response_handlers.py in process_storage_error (storage_error) 
 145 error . error_code = error_code
 146 error . additional_info = additional_data
 --> 147 raise error
 148 
 149 

 ResourceExistsError : The specified container already exists.
RequestId:1ab42782-c01e-000b-1072-5df5db000000
Time:2020-07-19T02:18:08.6436083Z
ErrorCode:ContainerAlreadyExists
Error:None

In [4]:
configs = {"fs.azure.account.auth.type": "OAuth",
           "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
           "fs.azure.account.oauth2.client.id": dbutils.secrets.get(scope="dgsecretscope",key="storagegen2mountappclientid"),
           "fs.azure.account.oauth2.client.secret": dbutils.secrets.get(scope="dgsecretscope",key="storagegen2mountappsecret"),
           "fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/{}/oauth2/token".format(dbutils.secrets.get(scope="dgsecretscope",key="storagegen2mountapptenantid"))}


In [5]:
if not any(mount.mountPoint == '/mnt/datagamz/{}'.format(tenant) for mount in dbutils.fs.mounts()):
  dbutils.fs.mount(source = "abfss://{}@dg{}lakestorage.dfs.core.windows.net/".format(tenant, env), mount_point = "/mnt/datagamz/{}".format(tenant),extra_configs = configs)

In [6]:
db_name = "dg_{}".format(tenant)
spark.sql("create database if not exists dg_{db_name}  LOCATION '/mnt/datagamz/{tenant}/data/databases/{db_name}'".format(tenant=tenant, db_name=db_name))

Out[8]: DataFrame[]

In [7]:
dbutils.fs.ls("/mnt/datagamz/testing/data")

Out[11]: [FileInfo(path='dbfs:/mnt/datagamz/testing/data/databases/', name='databases/', size=0),
 FileInfo(path='dbfs:/mnt/datagamz/testing/data/pb_datasets/', name='pb_datasets/', size=0),
 FileInfo(path='dbfs:/mnt/datagamz/testing/data/rawdata/', name='rawdata/', size=0)]

In [8]:

print("Creating Databricks cluster for tenant")
base_url = dbutils.secrets.get(scope="dgsecretscope", key="databricksinstanceurl")
url = base_url + "api/2.0/clusters/create"
headers = {'Authorization': 'Bearer {}'.format(dbutils.secrets.get(scope="dgsecretscope", key="dabaricksaccesstoken"))}
body = {
  "autoscale": {
      "min_workers": 1,
      "max_workers": 2
  },
  "cluster_name": "{}".format(tenant),
  "spark_version": "7.0.x-scala2.12",
  "spark_conf": {
      "spark.rpc.message.maxSize": "1024",
      "spark.databricks.session.share": "false",
      "spark.speculation": true
  },
  "node_type_id": "Standard_DS3_v2",
  "driver_node_type_id": "Standard_DS3_v2",
  "ssh_public_keys": [],
  "custom_tags": {},
  "cluster_log_conf": {
      "dbfs": {
          "destination": "dbfs:/mnt/dganalyics/{}/logs".format(tenant)
      }
  },
  "spark_env_vars": {},
  "autotermination_minutes": 30,
  "enable_elastic_disk": True
}


Creating Databricks cluster for tenant

In [9]:
import requests as rq
cluster_create = rq.post(url, headers=headers, json=body)

In [10]:
if cluster_create.status_code != 200:
  raise Exception

Out[15]: 200